In [ ]:
R1
#CGAATGCACTATTCATTTTATGGGGGTAATTTTTTGAGTAACCATTTTTACTTTTTTTTTACATATGCGCATACTTCGAGTTATACAATATTATT CAAATAAAACCACAAAAAAG GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCT
#CGAATGCACTATTCATTTTATGGGGGTAATTTTTTGAGTAACCATTTTTACTTTTTTTTTACATATGCGCATACTTCGAGTTATACAATATTATT AACGAAATAATGTTTTTGAC GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCT
#CGAATGCACTATTCATTTTATGGGGGTAATTTTTTGAGTAACCATTTTTACTTTTTTTTTACATATGCGCATACTTCGAGTTATACAATATTATT AACGAAATAATGTTTTTGAC GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCT
#CGAATGCACTATTCATTTTATGGGGGTAATTTTTTGAGTAACCATTTTTACTTTTTTTTTACATATGCGCATACTTCGAGTTATACAATATTATT AACGAAATAATGTTTTTGAC GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCT


In [8]:
bc_file

,mutant,barcode
0,PBANKA_1224200_G1,TTAGATATGCATACAAAACA
1,PBANKA_1224200_G2,TCTTGGAATTTGAATGTCAA
2,PBANKA_1225600_G1,CAAATAAAACCACAAAAAAG
3,PBANKA_1225600_G2,AATGAATTTCCCCTTTTTTG
4,PBANKA_0914500_G1,ATAACAATAATAATAATACA
5,PBANKA_0914500_G2,TACGAAATAAATTTCTTCAC
6,PBANKA_0622900_G1,TCCATAATACATATAGCAAA
7,PBANKA_0622900_G2,AACGAAATAATGTTTTTGAC
8,PBANKA_1451000_G1,AAAAAGTATGACCCTATAAG
9,PBANKA_1451000_G2,AAAAAATAAGGTATTTCAAT


In [18]:
import gzip
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path
import regex as re

def open_gzip_optional(fname,mode="rt"):
    if str(fname).endswith("gz"):
        return gzip.open(fname,mode)
    elif str(fname).endswith("bz2"):
        return bz2.open(fname,mode)
    else:
        return open(fname,mode)
            
def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in dna[::-1]])

rootdir = Path("/corgi/otherdataset/ellenbushell/phit/")
fastqdir = rootdir / "fastq"

################ Read barcode collection. barcode = sgRNA
#import pandas as pd
bc_file = pd.read_csv("/corgi/otherdataset/ellenbushell/phit/bc.csv",sep=" ")
bc_file.columns = ["mutant","barcode"]
dict_bc2mutant = dict(zip(bc_file["barcode"],bc_file["mutant"]))



################ Prepare parser
flank_regex = re.compile("ATTATT"+"(\w{18,30})"+"GTTT")

################ Process all files
#sample_dict = {}
list_samples = []
list_counts = []
for seq_file in fastqdir.iterdir():
    if str(seq_file).endswith("fastq.gz"):
        
        #if len(list_samples)>5:
        #    break


        ####### Extract sample name
        #P28011_1172_S11_L001_R1_001.fastq.gz
        # => P28011_1172
        #sample = format_filename(seq_file)
        sample = seq_file.name.split("_S")[0]
        
        #Empty counts dictionary
        count_dict = dict(zip(list(bc_file["mutant"]), [0]*bc_file.shape[0]))
        count_dict["_other"] = 0 
        count_dict["_err"] = 0 
        
        #Reverse complement if read2
        do_revcomp = ("_R2_" in str(seq_file))
        
        if "_R1_" in seq_file.name:
            sample = sample+":R1"
        else:
            sample = sample+":R2"
        print(sample)
        
        
        #For all lines
        with open_gzip_optional(seq_file) as f1:
            it1 = iter(f1)
            count = 0
            while True:
                try:
                    
                    #Read one entry in FASTQ
                    next(it1)
                    theseq = next(it1).strip()
                    next(it1)
                    next(it1)
                    
                    if do_revcomp:
                        theseq = reverse_complement(theseq)

                    #print("> "+theseq)
                        
                    #Parse one entry
                    try:
                        
                        putative_barcode = re.search(flank_regex, theseq)[1]
                        #print(putative_barcode)
                        
                        putative_barcode = putative_barcode  #for this input file
                        
                        if putative_barcode in dict_bc2mutant.keys():
                            count_dict[dict_bc2mutant[putative_barcode]] += 1
                        else:
                            count_dict["_other"] += 1
                            #print(str(count)+" not in dict "+putative_barcode)

                    except TypeError:
                        count_dict["_err"] += 1
                        #print(str(count)+" type err: "+theseq)
                    
                    count=count+1

                except StopIteration:
                    break
        
        #Turn into an array, add to the list
        list_samples.append(sample)
        list_counts.append(
            np.array([count_dict["_other"], count_dict["_err"]]+[count_dict[i] for i in bc_file["mutant"]])
        )
        
        

### Turn it all into one matrix
df = pd.DataFrame(
    np.vstack(list_counts).T,
    columns = list_samples, 
    index = ["_other","_err"]+list(bc_file["mutant"]))

P28011_1162:R1
P28011_1162:R2
P28011_1163:R1
P28011_1163:R2
P28011_1164:R1
P28011_1164:R2
P28011_1165:R1
P28011_1165:R2
P28011_1166:R1
P28011_1166:R2
P28011_1167:R1
P28011_1167:R2
P28011_1168:R1
P28011_1168:R2
P28011_1169:R1
P28011_1169:R2
P28011_1170:R1
P28011_1170:R2
P28011_1171:R1
P28011_1171:R2
P28011_1172:R1
P28011_1172:R2
P28011_1173:R1
P28011_1173:R2
P28011_1174:R1
P28011_1174:R2
P28011_1175:R1
P28011_1175:R2
P28011_1176:R1
P28011_1176:R2
P28011_1177:R1
P28011_1177:R2


In [19]:
df

,P28011_1162:R1,P28011_1162:R2,P28011_1163:R1,P28011_1163:R2,P28011_1164:R1,P28011_1164:R2,P28011_1165:R1,P28011_1165:R2,P28011_1166:R1,P28011_1166:R2,...,P28011_1173:R1,P28011_1173:R2,P28011_1174:R1,P28011_1174:R2,P28011_1175:R1,P28011_1175:R2,P28011_1176:R1,P28011_1176:R2,P28011_1177:R1,P28011_1177:R2
_other,9922,8903,16103,13141,19591,14200,16735,12329,16008,13737,...,10092,8984,7891,6737,7878,7072,6699,5580,8696,7775
_err,3255,3324,15673,18918,44528,49938,52640,57793,24166,26119,...,9731,9797,32998,32792,3676,3716,2929,2689,3529,3552
PBANKA_1224200_G1,28512,28489,6344,6338,2761,2751,8029,8007,23956,23941,...,30452,30407,50445,50452,35231,35094,25896,25954,57908,57813
PBANKA_1224200_G2,12363,12419,2036,2037,2813,2833,3729,3737,2739,2754,...,4767,4760,2766,2761,3881,3901,4255,4275,3263,3265
PBANKA_1225600_G1,9832,9847,818,828,683,691,110,110,949,952,...,85,85,116,117,97,98,22,23,60,61
PBANKA_1225600_G2,26314,26298,3160,3153,2930,2928,1714,1713,6493,6454,...,19132,19079,9339,9302,15130,15067,17506,17488,9160,9112
PBANKA_0914500_G1,17108,17194,1502,1498,343,343,1179,1175,425,428,...,14,14,15,15,2,3,8,8,2,1
PBANKA_0914500_G2,15536,15565,9824,9849,7185,7189,2573,2577,16723,16701,...,29605,29506,23585,23649,26578,26513,25173,25225,27190,27124
PBANKA_0622900_G1,27599,27686,4343,4364,1845,1855,2984,2976,10153,10178,...,21451,21422,14093,14173,18076,18100,16282,16301,13344,13339
PBANKA_0622900_G2,20502,20588,18628,18623,10019,10075,11575,11631,37115,37267,...,38265,38337,39440,39653,32532,32629,28463,28592,36659,36752


In [26]:
#Check how well it worked
df[0:2]/df.sum(axis=0)

#df[-1]/df.sum(axis=0)



,P28011_1162:R1,P28011_1162:R2,P28011_1163:R1,P28011_1163:R2,P28011_1164:R1,P28011_1164:R2,P28011_1165:R1,P28011_1165:R2,P28011_1166:R1,P28011_1166:R2,...,P28011_1173:R1,P28011_1173:R2,P28011_1174:R1,P28011_1174:R2,P28011_1175:R1,P28011_1175:R2,P28011_1176:R1,P28011_1176:R2,P28011_1177:R1,P28011_1177:R2
_other,0.042949,0.038538,0.086413,0.070518,0.078505,0.056902,0.067796,0.049947,0.070156,0.060203,...,0.038524,0.034294,0.031052,0.026511,0.035072,0.031484,0.031031,0.025848,0.035717,0.031935
_err,0.014090,0.014388,0.084105,0.101519,0.178432,0.200111,0.213254,0.234130,0.105909,0.114468,...,0.037146,0.037398,0.129852,0.129041,0.016365,0.016543,0.013568,0.012456,0.014495,0.014589


In [27]:
df.iloc[ -1]/df.sum(axis=0)


P28011_1162:R1    0.000255
P28011_1162:R2    0.000277
P28011_1163:R1    0.463805
P28011_1163:R2    0.460451
P28011_1164:R1    0.580719
P28011_1164:R2    0.579553
P28011_1165:R1    0.529792
P28011_1165:R2    0.525765
P28011_1166:R1    0.215051
P28011_1166:R2    0.213649
P28011_1167:R1    0.188569
P28011_1167:R2    0.187658
P28011_1168:R1    0.284248
P28011_1168:R2    0.279993
P28011_1169:R1    0.048197
P28011_1169:R2    0.046701
P28011_1170:R1    0.078249
P28011_1170:R2    0.077748
P28011_1171:R1    0.055018
P28011_1171:R2    0.054332
P28011_1172:R1    0.003743
P28011_1172:R2    0.003654
P28011_1173:R1    0.009589
P28011_1173:R2    0.009520
P28011_1174:R1    0.004230
P28011_1174:R2    0.004246
P28011_1175:R1    0.001349
P28011_1175:R2    0.001367
P28011_1176:R1    0.001742
P28011_1176:R2    0.001760
P28011_1177:R1    0.001253
P28011_1177:R2    0.001261
dtype: float64

In [28]:
df.to_csv(rootdir / "counts.csv")

#rootdir = Path("/corgi/otherdataset/ellenbushell/P28011/")
